<a href="https://colab.research.google.com/github/atikhasan007/Natural-Language-Processing./blob/main/text%20summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("nileshmalode1/samsum-dataset-text-summarization")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/samsum-dataset-text-summarization


In [29]:
import os

dataset_path = "/root/.cache/kagglehub/datasets/nileshmalode1/samsum-dataset-text-summarization/versions/1"
files = os.listdir(dataset_path)
files

['samsum_dataset',
 'samsum-train.csv',
 'samsum-test.csv',
 'samsum-validation.csv']

In [30]:
file_name_for_train = 'samsum-train.csv'
full_path_for_train = os.path.join(dataset_path,file_name_for_train)

In [31]:
file_name_for_validation ='samsum-validation.csv'
full_path_for_validation = os.path.join(dataset_path,file_name_for_validation)

In [32]:
import pandas as pd

train_data = pd.read_csv(full_path_for_train)

validation_data = pd.read_csv(full_path_for_validation)

In [33]:
train_data.head()


,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."


In [34]:
train_data.shape

(14732, 3)

In [35]:
validation_data.shape

(818, 3)

In [36]:
train_data = train_data.sample(n=4000,random_state=42).reset_index(drop=True)
validation_data = validation_data.sample(n=500,random_state=42).reset_index(drop=True)

In [37]:
train_data.shape

(4000, 3)

In [38]:
validation_data.shape

(500, 3)

In [39]:
#clean the text by removing unwanted characters
import re

def clean_text(text):
    text = re.sub(r'\r\n',' ', text) #remove carriage return and line breaks
    text = re.sub(r'\s+',' ',text) #Remove extra space
    text = re.sub(r'<.*?>','',text)#remove any xml tags
    text = text.strip().lower()
    return text

In [40]:
# Apply cleaning to dialogue and summary columns
train_data['dialogue'] = train_data['dialogue'].apply(clean_text)
train_data['summary'] = train_data['summary'].apply(clean_text)

validation_data['dialogue'] = validation_data['dialogue'].apply(clean_text)
validation_data['summary'] = validation_data['summary'].apply(clean_text)


In [41]:

# Display a sample after cleaning
train_data

,id,dialogue,summary
0,13811908,violet: hi! i came across this austin's articl...,violet sent claire austin's article.
1,13716431,pat: so does anyone know when the stream is go...,pat and lou are waiting for the stream but kev...
2,13810214,jane: jane: whaddya think? shona: this ur tin...,jane is updating her tinder profile tonight an...
3,13729823,"adam: do u have a map of paris? tom: yes, why?...",tom has a map of paris.
4,13681400,"frank: hi, how's the family? mike: great! sam'...","mike is happy, because sam's moved out. mike a..."
...,...,...,...
3995,13681041,barry: hello buddy michael: hey barry: do you ...,barry and michael will watch football instead ...
3996,13818705,karen: hey lisa. larissa and me have recently ...,karen and larissa moved to belgium and ask lis...
3997,13821859,"miles: hey, guys, i'm so sorry, but i missed t...","miles has missed the bus, so he may be 15 minu..."
3998,13812716,emma: did you finish the book i gave you? liam...,"emma gave ""the first fifteen lives of harry au..."


In [42]:
import pandas as pd
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments

In [57]:
# Text summarization

# Translation

# Question answering

# Text-to-text generation


In [43]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")



**Tokenizer**

In [44]:
input_max_len =  max(len(tokenizer.encode(text)) for text in train_data['dialogue'])
input_max_len

Token indices sequence length is longer than the specified maximum sequence length for this model (539 > 512). Running this sequence through the model will result in indexing errors


1224

In [45]:
output_max_len = max(len(tokenizer.encode(text)) for text in train_data['summary'])
output_max_len

103

In [46]:
#preprocessing function for tokenization
def preprocess_function(examples):
    #Tokenizer the dialogue and summary
    inputs = tokenizer(examples['dialogue'],padding='max_length',truncation=True,max_length=512)
    targets = tokenizer(examples['summary'],padding='max_length',truncation=True,max_length=150)
    inputs['labels'] = targets['input_ids']
    return inputs


In [47]:

# Apply the preprocessing
train_dataset = train_data.apply(preprocess_function, axis=1)
val_dataset = validation_data.apply(preprocess_function, axis=1)

In [48]:
train_dataset[0]

{'input_ids': [25208, 10, 7102, 55, 3, 23, 764, 640, 48, 403, 17, 77, 31, 7, 1108, 11, 3, 23, 816, 24, 25, 429, 253, 34, 1477, 25208, 10, 3, 7997, 15, 10, 7102, 55, 3, 10, 61, 2049, 6, 68, 3, 23, 31, 162, 641, 608, 34, 5, 3, 10, 61, 3, 7997, 15, 10, 68, 2049, 21, 1631, 81, 140, 3, 10, 61, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [50]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

# Model
model = T5ForConditionalGeneration.from_pretrained("t5-small")

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=6,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_steps=500,
    eval_steps=50,

)


# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model
trainer.train()


Step,Training Loss
50,14.632700
100,11.451900
150,5.288000
200,1.462900
250,0.714400
300,0.614900
350,0.505700
400,0.486400
450,0.437600
500,0.427900


TrainOutput(global_step=3000, training_loss=0.9067391357421875, metrics={'train_runtime': 1123.3472, 'train_samples_per_second': 21.365, 'train_steps_per_second': 2.671, 'total_flos': 3248203235328000.0, 'train_loss': 0.9067391357421875, 'epoch': 6.0})

In [51]:
#save and load model

# Save the fine-tuned model
model.save_pretrained("./saved_summary_model")
tokenizer.save_pretrained("./saved_summary_model")

('./saved_summary_model/tokenizer_config.json',
 './saved_summary_model/special_tokens_map.json',
 './saved_summary_model/spiece.model',
 './saved_summary_model/added_tokens.json')

In [52]:
# Load the saved model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("./saved_summary_model")
tokenizer = T5Tokenizer.from_pretrained("./saved_summary_model")

In [53]:
#summarization system

# Ensure the model is on the correct device (GPU if available)
device = model.device  # Get the device the model is on

def summarize_dialogue(dialogue):
    dialogue = clean_text(dialogue)  # Assuming clean_text is defined
    inputs = tokenizer(dialogue, return_tensors="pt", truncation=True, padding="max_length", max_length=512)

    # Move input tensors to the same device as the model
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Generate summary
    outputs = model.generate(
        inputs["input_ids"],
        max_length=150,
        num_beams=4,
        early_stopping=True
    )

    # Decode the generated summary
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

In [54]:
# Test with a sample input
sample_dialogue = """
Violet: Hey Claire! I was reading an article about Austin and thought you might find it interesting!
Violet: It's about the current trends in urban development and how cities are planning for the future.
Violet: Here, let me share the link: <file_other>
Claire: Oh wow, that sounds like an insightful read. But I've actually already read that one last week.
Claire: It was really interesting though, especially the part about sustainable architecture in cities.
Claire: You know, I've been following these urban planning discussions for a while now.
Violet: Oh, I didn’t know that! Well, I’ll look for something else then, maybe something about eco-friendly cities or tech innovations.
Claire: That would be awesome! Let me know if you find something cool.
Violet: Sure, I’ll keep you posted. Thanks for the feedback!
"""

summary = summarize_dialogue(sample_dialogue)
print("Summary:", summary)

Summary: claire was reading an article about austin and thought it might be interesting. he's already read that one last week. he's been following urban planning discussions for a while now.


In [55]:
#Download Mode To Your Machine
import shutil

# Path to the directory containing the fine-tuned model
model_dir = "./saved_summary_model"

# Output zip file path
output_zip_path = "saved_summary_model.zip"

# Create a zip archive
shutil.make_archive(base_name="saved_summary_model", format="zip", root_dir=model_dir)


'/content/saved_summary_model.zip'

In [56]:
from IPython.display import FileLink

# Display a download link
FileLink(output_zip_path)

/content/saved_summary_model.zip